# Begin

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("lrex").getOrCreate()

# Load training data
training = spark.read.format("libsvm")\
    .load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/sample_linear_regression_data-2.txt")


In [0]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:

lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction",)
                      #maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)



In [0]:
lrModel.coefficients

Out[4]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[5]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

training_summary.r2

Out[6]: 0.027839179518600154

In [0]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]
Intercept: 0.14228558260358093
numIterations: 0
objectiveHistory: [0.0]
+-------------------+
 residuals|
+-------------------+
-11.011130022096554|
 0.9236590911176538|
-4.5957401897776675|
 -20.4201774575836|
-10.339160314788181|
-5.9552091439610555|
-10.726906349283922|
 2.122807193191233|
 4.077122222293811|
-17.316168071241652|
 -4.593044343959059|
 6.380476690746936|
 11.320566035059846|
-20.721971774534094|
 -2.736692773777401|
 -16.66886934252847|
 8.242186378876315|
-1.3723486332690233|
-0.7060332131264666|
-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.163092
r2: 0.027839

In [0]:
all_data = spark.read.format("libsvm")\
    .load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/sample_linear_regression_data-2.txt")

In [0]:
train_data, test_data = all_data.randomSplit([0.7,0.3])


In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate( test_data )
test_results.rootMeanSquaredError

Out[11]: 10.19448293882429

In [0]:
unlabelled_data = test_data.select("features")
unlabelled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [0]:
predictions = correct_model.transform( unlabelled_data )
predictions.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...| 0.4754564004599843|
(10,[0,1,2,3,4,5,...| 1.2894922048246509|
(10,[0,1,2,3,4,5,...| 2.4147216966544924|
(10,[0,1,2,3,4,5,...| 1.2546801916397625|
(10,[0,1,2,3,4,5,...| 0.9727123204426934|
(10,[0,1,2,3,4,5,...|-1.8589348751366601|
(10,[0,1,2,3,4,5,...|0.15741803612471172|
(10,[0,1,2,3,4,5,...| 4.07863060777336|
(10,[0,1,2,3,4,5,...| 1.3496503457601565|
(10,[0,1,2,3,4,5,...| 1.1706787617832561|
(10,[0,1,2,3,4,5,...| 0.8461546275038174|
(10,[0,1,2,3,4,5,...|-0.6077063138375269|
(10,[0,1,2,3,4,5,...|-0.8844204037234185|
(10,[0,1,2,3,4,5,...| -1.84779755950228|
(10,[0,1,2,3,4,5,...|-2.4275019642422437|
(10,[0,1,2,3,4,5,...| 3.4346223666565527|
(10,[0,1,2,3,4,5,...| 0.722412126349913|
(10,[0,1,2,3,4,5,...| 0.7942257056646259|
(10,[0,1,2,3,4,5,...|-1.1528815912159789|
(10,[0,1,2,3,4,5,...|-1.4765219974415418|
+--------------------+-------------------+
only showing top 20 rows